In [1]:
import json
import pandas as pd
from pandas import json_normalize


In [2]:
# reading the JSON data using json.load()
file = 'focal_data.json'
with open(file) as focal_file:
    dict_focal = json.load(focal_file)


In [3]:
# converting json dataset from dictionary to dataframe
df = pd.DataFrame.from_dict(dict_focal, orient='columns')
#df.reset_index(level=0, inplace=True)

In [4]:
df2 = df.explode('references').reset_index(drop=True)

In [5]:
file = 'refer_data.json'
with open(file) as refer_file:
    dict_ref = json.load(refer_file)

In [6]:
# converting json dataset from dictionary to dataframe
df_ref = pd.DataFrame.from_dict(dict_ref, orient='columns')


In [7]:
df_ref.columns = ['ref_index', 'ref_abstract', 'ref_authors', 'ref_n_citation', 'ref_references', 'ref_title', 'ref_venue', 'ref_year', 'references']

In [8]:
df2['references'].astype(str)

0        6b19b1e6-81c5-4039-8a70-f3a2797f5447
1        8823b473-17c0-4f1f-a084-f86ae4b29e63
2        b9e545cf-0862-4ad2-873c-92740caca989
3        ea135772-baf4-4fca-b689-7e0bf2861a2b
4        11a9d4ce-e21c-4f59-bf52-e2cc3b52de45
                         ...                 
14266    a38af87a-9fb6-43e6-9264-6cd31aba1277
14267    c01ef5fd-4d9c-4b8a-b329-fe77e9ccf5ad
14268    ddaa5c0f-875c-49aa-8506-8fa2089c334d
14269    dfb54126-1662-4a1b-9f99-a1bf646f04a6
14270    e0e020ff-ac22-436e-ac8b-887490218ff6
Name: references, Length: 14271, dtype: object

In [9]:
df3 = pd.merge(df2, df_ref, on='references')

In [10]:
df3 = df3.astype({'abstract':str}) 
df3 = df3.astype({'ref_abstract': str})

In [11]:
df3

,_id,abstract,authors,n_citation,references,title,venue,year,id,ref_index,ref_abstract,ref_authors,ref_n_citation,ref_references,ref_title,ref_venue,ref_year
0,,This paper presents the design and implementat...,"[Sergio R. Geninatti, José Ignacio Benavides B...",0,6b19b1e6-81c5-4039-8a70-f3a2797f5447,Concurrent calculations on reconfigurable logi...,International Journal of Reconfigurable Computing,2010,9cee0c22-b065-4e23-925d-68c1cfc75dbe,,The combination of hardware acceleration and f...,"[Heather Quinn, Laurie A. Smith King, Miriam L...",50,"[115b9e24-0791-4251-8777-24c38fea952f, 430c493...",Runtime assignment of reconfigurable hardware ...,field programmable custom computing machines,2003
1,,This paper presents the design and implementat...,"[Sergio R. Geninatti, José Ignacio Benavides B...",0,8823b473-17c0-4f1f-a084-f86ae4b29e63,Concurrent calculations on reconfigurable logi...,International Journal of Reconfigurable Computing,2010,9cee0c22-b065-4e23-925d-68c1cfc75dbe,,This work presents a new scene change detectio...,"[Edmundo Saez, José Ignacio Benavides, Nicolás...",12,"[181e1f31-1580-454c-bdb8-7e46839d453d, 53febd2...",Reliable real time scene change detection in M...,international conference on multimedia and expo,2004
2,,This paper presents the design and implementat...,"[Sergio R. Geninatti, José Ignacio Benavides B...",0,b9e545cf-0862-4ad2-873c-92740caca989,Concurrent calculations on reconfigurable logi...,International Journal of Reconfigurable Computing,2010,9cee0c22-b065-4e23-925d-68c1cfc75dbe,,We discuss the use of Field Programmable Gate ...,"[Reid B. Porter, Kevin P. Mccabe, Neil W. Berg...",50,"[2633935f-57cc-40ad-9ee4-6dcb72aae169, 6a6b9aa...",An applications approach to evolvable hardware,,1999
3,,This paper presents the design and implementat...,"[Sergio R. Geninatti, José Ignacio Benavides B...",0,ea135772-baf4-4fca-b689-7e0bf2861a2b,Concurrent calculations on reconfigurable logi...,International Journal of Reconfigurable Computing,2010,9cee0c22-b065-4e23-925d-68c1cfc75dbe,,A parallel images processing field programmabl...,"[Takeaki Sugimura, JeoungChill Shim, Hiroyuki ...",7,[28f5ffcb-1076-49d0-9cf0-8a64694e2f67],Parallel image processing field programmable g...,field-programmable technology,2003
4,,In this paper an approach to the performance a...,"[Srđan Jovković, Stefan Panic, Mihajlo C. Stef...",50,11a9d4ce-e21c-4f59-bf52-e2cc3b52de45,Performance analysis of SSC diversity receiver...,Eurasip Journal on Wireless Communications and...,2010,a024dd21-83ad-4def-bfa5-8f714f502687,,This paper describes a method of deriving chan...,"[Klaus Witrisal, Yong-Ho Kim, Ramjee Prasad]",79,"[47b6b7ee-1947-453a-b664-e2e1f9f31def, 7291325...",A new method to measure parameters of frequenc...,IEEE Transactions on Communications,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14266,,The main contribution of this work is to propo...,"[Marcos F. Caetano, Jacir Luiz Bordim]",50,a38af87a-9fb6-43e6-9264-6cd31aba1277,A Cluster Based Collaborative Cache Approach f...,international conference on networking,2010,d34dae3a-a13c-4f26-bf56-f1d8b80c3d38,,Caching of frequently accessed data in multi-h...,"[Narottam Chand, Ramesh C. Joshi, Manoj Misra]",19,"[0d4d0363-07b5-43b6-976d-955e96044709, 194fd57...",Efficient Cooperative Caching in Ad Hoc Networks,communication system software and middleware,2006
14267,,The main contribution of this work is to propo...,"[Marcos F. Caetano, Jacir Luiz Bordim]",50,c01ef5fd-4d9c-4b8a-b329-fe77e9ccf5ad,A Cluster Based Collaborative Cache Approach f...,international conference on networking,2010,d34dae3a-a13c-4f26-bf56-f1d8b80c3d38,,Data caching on mobile clients is widely seen ...,"[Wenzhong Li, Edward Chan, Daoxu Chen]",15,"[01b1f5bb-f26f-4691-8da0-d0d184c47fa5, 01dffae...",Energy-Efficient Cache Replacement Policies fo...,wireless communications and networking conference,2007
14268,,The main contribution of this work is to propo...,"[Marcos F. Caetano, Jacir Luiz Bor

In [12]:
#similarity score as a new column 
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
vectorizer = CountVectorizer()

In [13]:
vector_similarity =[]
for index, row in df3.iterrows():
    allsentences = [row['abstract'], row['ref_abstract']]
    all_sentences_to_vector = vectorizer.fit_transform(allsentences)
    text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
    text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()
    cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
    vector_similarity.append(round((1-cosine)*100,2))

In [14]:
df3['vector_similarity'] = vector_similarity


#Glove embedding
gloveFile = "glove.6B.50d.txt"
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    with open(gloveFile, encoding="utf8" ) as f:
        content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model

import re
from nltk.corpus import stopwords
import pandas as pd

def preprocess(raw_text):

    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split 
    words = letters_only_text.lower().split()

    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))

    return cleaned_words

def cosine_distance_between_two_words(word1, word2):
    import scipy
    return (1- scipy.spatial.distance.cosine(model[word1], model[word2]))

def calculate_heat_matrix_for_two_sentences(s1,s2):
    s1 = preprocess(s1)
    s2 = preprocess(s2)
    result_list = [[cosine_distance_between_two_words(word1, word2) for word2 in s2] for word1 in s1]
    result_df = pd.DataFrame(result_list)
    result_df.columns = s2
    result_df.index = s1
    return result_df

def cosine_distance_wordembedding_method(s1, s2):
    import scipy
    vector_1 = np.mean([model[word] for word in preprocess(s1)],axis=0)
    vector_2 = np.mean([model[word] for word in preprocess(s2)],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    print('Word Embedding method with a cosine distance asses that our two sentences are similar to',round((1-cosine)*100,2),'%')


import scipy
model = loadGloveModel(gloveFile)


glove_similarity =[]
for i, r in df3.iterrows():
    s1 = r['abstract']
    s2 = r['ref_abstract']
    #print("--FOCAL--" + row['abstract'] + "--REFERENCE--" + row['ref_abstract'])
    s1 = preprocess(s1)
    s2 = preprocess(s2)
    for word1 in s1:
        vector_1 = np.mean(model[word1], axis=0)
    for word2 in s2:
        vector_2 = np.mean(model[word2], axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    glove_similarity.append(round((1-cosine)*100,2))

In [15]:
#glove_similarity

In [16]:
#ADD NEW COLUMNS 1. NUMBER OF AUTHORS, 2. NUMBER OF REFERENCES, 3. SIMILARITY SCORES (2) 
#4. LATEST REFERENCED YEAR 5. MEDIAN REFERENCE YEAR 5. NUMBER OF CITATIONS OF REFERENCES

mean_similarity_count = pd.pivot_table(df3, values='vector_similarity', index=['id'], columns=None, aggfunc='mean')
max_similarity_count = pd.pivot_table(df3, values='vector_similarity', index=['id'], columns=None, aggfunc='max')

In [17]:
mean_similarity_count
max_similarity_count

,vector_similarity
id,
005b0399-86d1-4c0e-97ae-a1e7b8d1863d,77.29
005f82a5-0399-4223-a43d-0ec531cf2128,68.60
00b70048-595b-438a-bc77-2e6cf0a6cfb7,50.77
00f351f3-bc9d-4fea-8b1a-9da33e8d8bac,47.32
00fb8632-b5d6-4635-bb8a-32253e277d57,56.30
...,...
ff5ed562-cc3d-4509-af45-211f21858069,51.26
ff664d2d-e7fd-4263-9e4e-50755aed848b,55.69
ff6d02dd-ed63-4fe2-b4c0-2536b0baa451,69.23


In [18]:
df_main = pd.merge(df, mean_similarity_count,  on='id')

In [19]:
df_main = pd.merge(df_main, max_similarity_count,  on='id')

In [20]:
avg_ref_year = pd.pivot_table(df3, values='ref_year', index=['id'], columns=None, aggfunc='mean')

In [21]:
max_ref_year = pd.pivot_table(df3, values='ref_year', index=['id'], columns=None, aggfunc='max')

In [22]:
df_main = pd.merge(df_main, avg_ref_year ,  on='id')

In [23]:
df_main = pd.merge(df_main, max_ref_year ,  on='id')

In [24]:
df_main.rename(columns = {'vector_similarity_x':'mean_vector_similarity', 'vector_similarity_y':'max_vector_similarity',
                         'ref_year_x': 'mean_ref_year', 'ref_year_y': 'max_ref_year' }, inplace = True)

In [25]:
df3

,_id,abstract,authors,n_citation,references,title,venue,year,id,ref_index,ref_abstract,ref_authors,ref_n_citation,ref_references,ref_title,ref_venue,ref_year,vector_similarity
0,,This paper presents the design and implementat...,"[Sergio R. Geninatti, José Ignacio Benavides B...",0,6b19b1e6-81c5-4039-8a70-f3a2797f5447,Concurrent calculations on reconfigurable logi...,International Journal of Reconfigurable Computing,2010,9cee0c22-b065-4e23-925d-68c1cfc75dbe,,The combination of hardware acceleration and f...,"[Heather Quinn, Laurie A. Smith King, Miriam L...",50,"[115b9e24-0791-4251-8777-24c38fea952f, 430c493...",Runtime assignment of reconfigurable hardware ...,field programmable custom computing machines,2003,50.53
1,,This paper presents the design and implementat...,"[Sergio R. Geninatti, José Ignacio Benavides B...",0,8823b473-17c0-4f1f-a084-f86ae4b29e63,Concurrent calculations on reconfigurable logi...,International Journal of Reconfigurable Computing,2010,9cee0c22-b065-4e23-925d-68c1cfc75dbe,,This work presents a new scene change detectio...,"[Edmundo Saez, José Ignacio Benavides, Nicolás...",12,"[181e1f31-1580-454c-bdb8-7e46839d453d, 53febd2...",Reliable real time scene change detection in M...,international conference on multimedia and expo,2004,40.34
2,,This paper presents the design and implementat...,"[Sergio R. Geninatti, José Ignacio Benavides B...",0,b9e545cf-0862-4ad2-873c-92740caca989,Concurrent calculations on reconfigurable logi...,International Journal of Reconfigurable Computing,2010,9cee0c22-b065-4e23-925d-68c1cfc75dbe,,We discuss the use of Field Programmable Gate ...,"[Reid B. Porter, Kevin P. Mccabe, Neil W. Berg...",50,"[2633935f-57cc-40ad-9ee4-6dcb72aae169, 6a6b9aa...",An applications approach to evolvable hardware,,1999,44.90
3,,This paper presents the design and implementat...,"[Sergio R. Geninatti, José Ignacio Benavides B...",0,ea135772-baf4-4fca-b689-7e0bf2861a2b,Concurrent calculations on reconfigurable logi...,International Journal of Reconfigurable Computing,2010,9cee0c22-b065-4e23-925d-68c1cfc75dbe,,A parallel images processing field programmabl...,"[Takeaki Sugimura, JeoungChill Shim, Hiroyuki ...",7,[28f5ffcb-1076-49d0-9cf0-8a64694e2f67],Parallel image processing field programmable g...,field-programmable technology,2003,39.51
4,,In this paper an approach to the performance a...,"[Srđan Jovković, Stefan Panic, Mihajlo C. Stef...",50,11a9d4ce-e21c-4f59-bf52-e2cc3b52de45,Performance analysis of SSC diversity receiver...,Eurasip Journal on Wireless Communications and...,2010,a024dd21-83ad-4def-bfa5-8f714f502687,,This paper describes a method of deriving chan...,"[Klaus Witrisal, Yong-Ho Kim, Ramjee Prasad]",79,"[47b6b7ee-1947-453a-b664-e2e1f9f31def, 7291325...",A new method to measure parameters of frequenc...,IEEE Transactions on Communications,2001,51.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14266,,The main contribution of this work is to propo...,"[Marcos F. Caetano, Jacir Luiz Bordim]",50,a38af87a-9fb6-43e6-9264-6cd31aba1277,A Cluster Based Collaborative Cache Approach f...,international conference on networking,2010,d34dae3a-a13c-4f26-bf56-f1d8b80c3d38,,Caching of frequently accessed data in multi-h...,"[Narottam Chand, Ramesh C. Joshi, Manoj Misra]",19,"[0d4d0363-07b5-43b6-976d-955e96044709, 194fd57...",Efficient Cooperative Caching in Ad Hoc Networks,communication system software and middleware,2006,58.77
14267,,The main contribution of this work is to propo...,"[Marcos F. Caetano, Jacir Luiz Bordim]",50,c01ef5fd-4d9c-4b8a-b329-fe77e9ccf5ad,A Cluster Based Collaborative Cache Approach f...,international conference on networking,2010,d34dae3a-a13c-4f26-bf56-f1d8b80c3d38,,Data caching on mobile clients is widely seen ...,"[Wenzhong Li, Edward Chan, Daoxu Chen]",15,"[01b1f5bb-f26f-4691-8da0-d0d184c47fa5, 01dffae...",Energy-Efficient Cache Replacement Policies fo...,wireless communications and networking conference,2007,51.50
14268,,The main contribution

mean_ref_citations = pd.pivot_table(df3, values='ref_n_citation', index=['id'], columns=None, aggfunc='mean')
df_main = pd.merge(df_main, mean_ref_citations,  on='id')

max_ref_citations = pd.pivot_table(df3, values='ref_n_citation', index=['id'], columns=None, aggfunc='max')
df_main = pd.merge(df_main, max_ref_citations,  on='id')

In [28]:
df_main.rename(columns = {'ref_n_citation_x':'mean_ref_n_citation', 'ref_n_citation_y':'max_ref_n_citation'}, inplace = True)

In [29]:
df_main

,_id,abstract,authors,n_citation,references,title,venue,year,id,mean_vector_similarity,max_vector_similarity,mean_ref_year,max_ref_year,mean_ref_n_citation,max_ref_n_citation
0,,This paper presents the design and implementat...,"[Sergio R. Geninatti, José Ignacio Benavides B...",0,"[6b19b1e6-81c5-4039-8a70-f3a2797f5447, 8823b47...",Concurrent calculations on reconfigurable logi...,International Journal of Reconfigurable Computing,2010,9cee0c22-b065-4e23-925d-68c1cfc75dbe,43.820000,50.53,2002.250000,2004,29.750000,50
1,,In this paper an approach to the performance a...,"[Srđan Jovković, Stefan Panic, Mihajlo C. Stef...",50,"[11a9d4ce-e21c-4f59-bf52-e2cc3b52de45, 337418c...",Performance analysis of SSC diversity receiver...,Eurasip Journal on Wireless Communications and...,2010,a024dd21-83ad-4def-bfa5-8f714f502687,68.175000,83.16,2005.000000,2010,81.625000,249
2,,BitTorrent (BT) is currently the main P2P prot...,"[Marios Iliofotou, Georgos Siganos, Xiaoyuan Y...",50,"[1fbf5388-cffe-4abc-b809-c9119bf9f39c, 2cd1e1c...",Comparing BitTorrent clients in the wild: the ...,international workshop on peer to peer systems,2010,791d78fe-cfeb-4fc8-8410-8fdab0951e29,54.667500,63.48,2007.500000,2009,291.000000,997
3,,This paper describes an approach for mobile ro...,"[Friedrich Fraundorfer, Changchang Wu, Marc Po...",2,"[1c09ef91-f69a-43e4-aaa7-571f3a143ea9, 6c38b3b...",Methods for combined monocular and stereo mobi...,,2010,77b90a8e-eeba-4a1e-b080-003a37a3c33d,49.249000,99.28,2004.600000,2010,4970.600000,42508
4,,High-order neural networks can be considered a...,"[Yuxin Ding, Li Dong, Ling Wang, Guohua Wu]",4,"[0ebb0b84-824a-47c4-8daf-3ef005e1aafc, 13058fc...",A high order neural network to solve crossbar ...,international conference on neural information...,2010,fd872163-c207-4b9b-a1bb-fc5037c120e6,43.100769,53.97,2005.230769,2009,71.769231,269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2075,,We present an AI called Visuo that guesses qua...,"[Jim Davies, Jonathan Gagné]",4,"[8876115b-93b8-4f61-b5b2-46e9e273b74b, 930b067...",Estimating quantitative magnitudes using seman...,national conference on artificial intelligence,2010,3272d265-e29d-4e5b-bb72-6e4042c2cbe0,24.118000,28.57,2003.400000,2007,1294.200000,2889
2076,,Due to the required high precision and reliabi...,"[Zein Salah, Bernhard Preim, Amir Samii, Rudol...",0,"[217c1c93-67b0-4adb-97ae-d36daad58a3d, 31364f4...",Enhanced Intraoperative Visualization for Brai...,,2010,a53ce1f4-8b58-4975-86ca-b9859d95f9b9,52.867500,57.84,1997.750000,2002,1045.750000,3587
2077,,Background. Literal or theoretical replication...,"[A. César C. França, Paulo Rupino Cunha, Fabio...",50,"[64ae4179-ed47-4a71-bee0-5e2fbf93fec7, 881824f...",The effect of reasoning strategies on success ...,,2010,f22cc843-5d9c-42cf-af3a-564b9e57f7b2,49.317500,61.87,2003.250000,2008,112.250000,240
2078,,This paper investigates how readily an unsolva...,"[Susan L. Epstein, Xi Yun]",6,"[0067923a-e1ea-4789-a58c-d83620ae39b1, 1c3f3b5...",From unsolvable to solvable: an exploration of...,national conference on artificial intelligence,2010,fb9bf14f-7230-48a6-8503-e7ca35864251,48.522308,60.65,2002.000000,2009,187.461538,473


In [30]:
#add title word count as a column
title_len =[]
for index, row in df_main.iterrows():
    string = row['title']
    title_len.append(len(string.split()))

df_main['title_len'] = title_len

In [31]:
#add number of the authors
n_authors = []
for index, row in df_main.iterrows():
    liste = row['authors']
    n_authors.append(len(liste))

df_main['n_authors'] = n_authors

In [32]:
# n_authors, n_reference

n_references = pd.pivot_table(df3, values='references', index=['id'], columns=None, aggfunc='count')
df_main = pd.merge(df_main, n_references,  on='id')
df_main.rename(columns = {'references_y':'n_references'}, inplace = True)

df_main

# #of words in abstract (longer the more citations)

ab_words = []
for index, row in df_main.iterrows():
    string = row['abstract']
    ab_words.append(len(string.split()))

df_main['ab_len'] = ab_words

In [35]:
#journal impact factor ask if I can have an access to Journal Citation Reports (JCR) database

df_main


,_id,abstract,authors,n_citation,references_x,title,venue,year,id,mean_vector_similarity,max_vector_similarity,mean_ref_year,max_ref_year,mean_ref_n_citation,max_ref_n_citation,title_len,n_authors,n_references,ab_len
0,,This paper presents the design and implementat...,"[Sergio R. Geninatti, José Ignacio Benavides B...",0,"[6b19b1e6-81c5-4039-8a70-f3a2797f5447, 8823b47...",Concurrent calculations on reconfigurable logi...,International Journal of Reconfigurable Computing,2010,9cee0c22-b065-4e23-925d-68c1cfc75dbe,43.820000,50.53,2002.250000,2004,29.750000,50,13,4,4,107
1,,In this paper an approach to the performance a...,"[Srđan Jovković, Stefan Panic, Mihajlo C. Stef...",50,"[11a9d4ce-e21c-4f59-bf52-e2cc3b52de45, 337418c...",Performance analysis of SSC diversity receiver...,Eurasip Journal on Wireless Communications and...,2010,a024dd21-83ad-4def-bfa5-8f714f502687,68.175000,83.16,2005.000000,2010,81.625000,249,17,5,8,109
2,,BitTorrent (BT) is currently the main P2P prot...,"[Marios Iliofotou, Georgos Siganos, Xiaoyuan Y...",50,"[1fbf5388-cffe-4abc-b809-c9119bf9f39c, 2cd1e1c...",Comparing BitTorrent clients in the wild: the ...,international workshop on peer to peer systems,2010,791d78fe-cfeb-4fc8-8410-8fdab0951e29,54.667500,63.48,2007.500000,2009,291.000000,997,11,4,8,189
3,,This paper describes an approach for mobile ro...,"[Friedrich Fraundorfer, Changchang Wu, Marc Po...",2,"[1c09ef91-f69a-43e4-aaa7-571f3a143ea9, 6c38b3b...",Methods for combined monocular and stereo mobi...,,2010,77b90a8e-eeba-4a1e-b080-003a37a3c33d,49.249000,99.28,2004.600000,2010,4970.600000,42508,9,3,10,75
4,,High-order neural networks can be considered a...,"[Yuxin Ding, Li Dong, Ling Wang, Guohua Wu]",4,"[0ebb0b84-824a-47c4-8daf-3ef005e1aafc, 13058fc...",A high order neural network to solve crossbar ...,international conference on neural information...,2010,fd872163-c207-4b9b-a1bb-fc5037c120e6,43.100769,53.97,2005.230769,2009,71.769231,269,10,4,13,118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2075,,We present an AI called Visuo that guesses qua...,"[Jim Davies, Jonathan Gagné]",4,"[8876115b-93b8-4f61-b5b2-46e9e273b74b, 930b067...",Estimating quantitative magnitudes using seman...,national conference on artificial intelligence,2010,3272d265-e29d-4e5b-bb72-6e4042c2cbe0,24.118000,28.57,2003.400000,2007,1294.200000,2889,6,2,5,69
2076,,Due to the required high precision and reliabi...,"[Zein Salah, Bernhard Preim, Amir Samii, Rudol...",0,"[217c1c93-67b0-4adb-97ae-d36daad58a3d, 31364f4...",Enhanced Intraoperative Visualization for Brai...,,2010,a53ce1f4-8b58-4975-86ca-b9859d95f9b9,52.867500,57.84,1997.750000,2002,1045.750000,3587,10,5,4,118
2077,,Background. Literal or theoretical replication...,"[A. César C. França, Paulo Rupino Cunha, Fabio...",50,"[64ae4179-ed47-4a71-bee0-5e2fbf93fec7, 881824f...",The effect of reasoning strategies on success ...,,2010,f22cc843-5d9c-42cf-af3a-564b9e57f7b2,49.317500,61.87,2003.250000,2008,112.250000,240,19,3,4,140
2078,,This paper investigates how readily an unsolva...,"[Susan L. Epstein, Xi Yun]",6,"[0067923a-e1ea-4789-a58c-d83620ae39b1, 1c3f3b5...",From unsolvable to solvable: an exploration of...,national conference on artificial intelligence,2010,fb9bf14f-7230-48a6-8503-e7ca35864251,48.522308,60.65,2002.000000,2009,187.461538,473,9,2,13,78


In [36]:
#save df3 as df_with_reference
df3.to_pickle("df_with_reference.pkl")
    

In [37]:
#save df_main as df_main
df_main.to_pickle("df_main.pkl")